### 필요한 패키지 임포트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import librosa
import librosa.display
import warnings
warnings.filterwarnings('ignore')
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kurtosis
from scipy.stats import skew
import csv
import time
import datetime

### 파일 경로 불러오기

In [3]:
# 파일 경로
#audio_path = "다운로드 된 파일 저장된 폴더 경로"
audio_path = "/content/drive/MyDrive/Dataset/downloads/wav" # 원본 wav 파일

### 함수 정의

In [4]:
def song_names(audio_path):               # audio_path : 음원 파일들의 폴더 경로
    files = list(os.listdir(audio_path))

    songs = []
    paths = []

    for f in files:
        song = f
        path = os.path.join(audio_path, f)
        songs.append(song)
        paths.append(path)

    return songs, paths

In [ ]:
songs, paths = song_names(audio_path)

In [ ]:
print(songs[:3], paths[:3])

['이 밤-양다일.wav', '희재-성시경.wav', '아니면서-성시경.wav'] ['/content/drive/MyDrive/Dataset/downloads/wav/이 밤-양다일.wav', '/content/drive/MyDrive/Dataset/downloads/wav/희재-성시경.wav', '/content/drive/MyDrive/Dataset/downloads/wav/아니면서-성시경.wav']


In [5]:
# soundwave
def soundwave(paths):

    audio_files = []
    song_name = []

    for path in paths:
        try:
            sn = path.split("/")[-1]
            y, sr = librosa.load(path)
            y, _ = librosa.effects.trim(y)
            song_name.append(sn)
            audio_files.append(y)
        except ValueError as val:
            print(path, val)

    return audio_files, song_name, sr               # sr 디폴트 : 22050 / 변경 가능한 sr 수치 : 44100

In [ ]:
audio_files, song_name, sr = soundwave(paths[:10])

In [ ]:
print(song_name)

['이 밤-양다일.wav', '희재-성시경.wav', '아니면서-성시경.wav', 'Falling-존박.wav', '화신 (花信)-박효신.wav', '첫째 날 (With 나얼)-Ecobridge.wav', '내려놔요-브라운 아이드 소울.wav', '말꼬리-윤종신.wav', '그리고 그려요-Eye To Eye.wav', '지나간다 (Piano Ver.)-김범수.wav']


In [6]:
# Features
def get_features(y, sr):
    # Features to concatenate in the final dictionary
    features = {'centroid': None, 'roloff': None, 'flux': None, 'rmse': None,
                'zcr': None, 'contrast': None, 'bandwidth': None, 'flatness': None, 'chroma_stft': None}
    
    # Count silence
    if 0 < len(y):
        y_sound, _ = librosa.effects.trim(y)
    features['sample_silence'] = len(y) - len(y_sound)

    # Using librosa to calculate the features
    features['chroma_stft']=librosa.feature.chroma_stft(y=y, sr=sr).ravel()
    features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr).ravel()
    features['roloff'] = librosa.feature.spectral_rolloff(y, sr=sr,).ravel()
    features['zcr'] = librosa.feature.zero_crossing_rate(y).ravel()
    features['rmse'] = librosa.feature.rms(y).ravel()
    features['flux'] = librosa.onset.onset_strength(y=y, sr=sr).ravel()
    features['contrast'] = librosa.feature.spectral_contrast(y, sr=sr).ravel()
    features['bandwidth'] = librosa.feature.spectral_bandwidth(y, sr=sr).ravel()
    features['flatness'] = librosa.feature.spectral_flatness(y).ravel()
    
    # harmony, perceptral treatment (+@ 부분)
    harm, perc = librosa.effects.hpss(y)
    features['harm'] = harm.ravel()
    features['perc'] = perc.ravel()

    # MFCC treatment
    mfcc = librosa.feature.mfcc(y, sr=sr)   # n_mfcc=20 (default) 
    for idx, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(idx)] = v_mfcc.ravel()
        
    # Get statistics from the vectors
    def get_moments(descriptors):
        result = {}
        for k, v in descriptors.items():
            result['{}_max'.format(k)] = np.max(v)
            result['{}_min'.format(k)] = np.min(v)
            result['{}_mean'.format(k)] = np.mean(v)
            result['{}_std'.format(k)] = np.std(v)
            result['{}_kurtosis'.format(k)] = kurtosis(v)
            result['{}_skew'.format(k)] = skew(v)
        return result
    
    dict_agg_features = get_moments(features)
#    onset_env = librosa.onset.onset_strength(y=audio_file, sr=sr)
#    dict_agg_features['tempo'] = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
    dict_agg_features['tempo'] = librosa.beat.tempo(y, sr=sr)[0]
    
    return dict_agg_features

In [7]:
# Extract features 
def final_extraction(dataset_dir=audio_path):
    songs, paths = song_names(audio_path)
    audio_files, song_name, sr = soundwave(paths)

    last_features = []

    for audio in audio_files:
        features = get_features(audio, sr)
        last_features.append(features)

    return last_features, song_name

In [ ]:
%%time

last_features, song_name = final_extraction()

In [ ]:
# len(last_features), len(song_name)

In [ ]:
def df_all_features(last_features, songs):
    df_features = pd.DataFrame(last_features)
    df_songs = pd.DataFrame(song_name, columns=['song'])

    return pd.concat([df_songs, df_features], axis=1)

In [ ]:
df_all_features = df_all_features(last_features, song_name)
df_all_features.head()

In [ ]:
# export the data
filename = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
df_all_features.to_csv(f'/content/drive/Othercomputers/내 노트북/Project/Models/Music_Genre_Classification_Recommendation/Feature_Result_csv/{filename}.csv')

In [ ]:
# import the data
df_all_features = pd.read_csv(f'/content/drive/Othercomputers/내 노트북/Project/Models/Music_Genre_Classification_Recommendation/Feature_Result_csv/{filename}.csv', index_col = 0)

In [ ]:
df_all_features

### 특정값 포함하는 데이터 추출 (데이터 분포 확인용)

In [ ]:
contains_metal = df_all_features['song'].str.contains("metal")
subset_df = df_all_features[contains_metal]

In [ ]:
subset_df['song'].count()

77